Python has two standard ways of getting a string representation of an object.

`repr()` - String representing the object as the developer wants to see it

`str()` - String representing the object as th euser wants to see it

These call the special methods `__repr__` and `__str__`

In [ ]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        # converting to a float catches bad input early on
        self.x = float(x)
        self.y = float(y)
    
    # having a iter special method makes it iterable, allowing for variable unpacking.
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    # {!r} calls the __repr__ formatter
    # while using {!s} would call the __str__ formatter
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
        
    def __str__(self):
        return str(tuple(self))
        
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
        bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return typle(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    # if there is method for bytes represetnation
    # naturally, there shoul be a method that imports from bytes
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(cotets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)
        
    def __format__(self, fmt_spec=''):
        components = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*components)

ord() return unicode code point of a character

bin() retur  binary representation of a unicode code point

chr() returns character of given unicode code point

## classmethod Versus staticmethod

###### classmethod
Defines a method that operates on the class, and not the instance.
Notices how it recieves the class itself, instead of the instance as the first argument - `cls` instead `self`.

It's most commonly used for alternative constructors, like frombytes. See how the return statement calls `cls`, to construct a new instance.

###### staticmethod
Changes a class method so that it does not recieve the special first argument - no `self` or `class`.
A staticmethod in Python is essentially a plain function that happens to be in the class body, and not in the module.

## Formatted Displays

format() built in and str.format() call .\__format__(format_spec)

Formatting specifier is either the second arg in format(obj, format_spec), or whatever appears after the colon in a {} replacement field for str.format

In [ ]:
print(format(0.1256512345, '0.4f'))
print('{rate:0.4f}'.format(rate=0.12513451))

Notation used in format specifier is called the Format Specification Mini-Language

In [ ]:
def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self)
    return '({}, {})'.format(*components)

To start to handle more complex format_specifier, add in some control flow:

In [ ]:
def __format__(self, fmt_spec=''):
    if fmt_spec.enswith('p'): # if string endswith p, use polar coordinates
        fmt_spec = fmt_spec[:-1] # remove p suffix from format_spec, allowing for applying other format rules.
        coords = (abs(self), self.angle())
        outer_fmt = '<{}, {}>'
    else:
        coords = self
        outer_fmt = '({}, {})'
    componetns = (format(c, fmt_spec) for c in coords) # generate iterable with formatted strings
    return outer_fmt.format(*components) # plug the iterable into the given outer_fmt

## A Hashable Vector2d

Until now, the vector instances are unhashable, which means they can't be put in a set.

So we need to implement \__hash__, and \__eq__(which we already have). Vector instances also need to be made immutable.

###### What is hashable? 
    An object is hashable if it has a hash which does not chance over its lifetime. It needs a \__hash__ and a \__eq__ method. atomic, immuteble types such as str, bytes, and numeric types are hashable. A tuple is hashable only if its elements are. 

To make a class instance immutable, its properties need to be made read only. You shouldn't be able to just do something like `v1.x = 7`

In [ ]:
class Vectord2:
    typecode = 'd'
    
    # use two leading underscores to make a attribute private
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
        
    # property decorater marks the getter method of a property
    @property
    def x(self):
        return self.__x
        
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))

Now we must create a \__hash__ method. Ideally, we combine the attribtues of the class and their hashes. It's recommended to use the XOR ^ operator.

In [ ]:
def __hash__(self):
    return hash(self.x) ^ hash(self.y)

If you're creating a type with a clear numeric value, it also makes sense to implement \__int__ and \__float__ methods, which are invoked by the int() and float() builtins.

__remember__, customers don't care if your objects are "Pythonic" 

## Private and "Protected" Attributes in Python

As Python does not have dedicated keywords for scope modifying like `private` in Java, name mangling is used instead.

If you name an instance variable with two leading underscores, `__attr`, Python stores the name in the instance's `__dict__` prefixed with a leading underscore and the class name. So the `__name` attr of class `Person` is stored as `_Person__name`.

So, in practise, you could do `instance._classname_attr` to access a private attribute. But this is not meant to be!

Consider name mangling like having a cover over a switch. It's a safety device, not a security one. It prevents accidental use, but does not prevent malicious use.

## Saving Space with __slots__ Class Attribute

Python stores instance attributes in aa per-instance dict named \__dict__. Dictionaries are not very memory efficient because of the underlying hash table used for fast access.


If you're dealing with many many instances, with few attributes, use the \__slots__ class attribute to save memory. \__slots__ lets the interpreter store instance attributes in a tuple instead of a dict.

\__slots__ attribute  cannot be inherited. Python only takes into account \__slots__ attribtues defined in each individual class.

In [ ]:
class Vector2d:
    # to define __slots__, create a variable with that name,
    # and assign it to an iterable of string representations of instance attribtues.
    __slots__ = ('__x', '__y')

However, if you're handling millions of objects with numeric data, then use NumPy arrays - which are extremely effecient

\__slots__ should only be used for optimization

CAreless optimization is worse than premature optimization

\__weakref__ attributes is necessary for an object to support weak references. This attribute is present by default in UDCs. Though you made need to include "\__weakref__" in "\__slots__", to allow for the instance to be weakly referenced.

It's likely that just using Pandas, wich is already optimised, is better.

The added complexity of using "\__slots__" is only really worth it if you need to memory effeciency.

## Overriding Class Attributes

In Python, class attributes can be used as default values for instance attributes.

In Vectord2d, we define the class attribute `typecode`. When you call `self.typecode`, as there is no instance variable called `typecode`, `Vector2d.typecode` is used by default.

Subclasses can override their parent's class variables:

In [ ]:
#Simply inherit Vector2d, change the typecode class variable, and keep all the other functionality.
class ShortVector2d(Vector2d):
    typecode = 'f'

"Simple is better than complex". A python object should be as simple as the requirements allow.

## Chapter summary

To build Python objects, observe how real Python objects behave

Python's design allows you to start with public attributes, and then make them private properties if the need arises. Python makes it much easier to follow the answer to "What's the simplest thing that could possibly work?"

Java's private and protected keywords are also more safety than security. You can still just get the private field using Java's reflection API. The application would require a SecurityManager, which rarely happens.

In [ ]:
s